# **基于MindSpore框架的AlexNet案例实现**


<div align=center>
    <img src="./image/banner.png" style="zoom:35%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;"></div>
</div>

### **1 论文讲解**
> 论文：《ImageNet Classification with Deep Convolutional Neural Networks》
> 
> 作者：Alex Krizhevsky，Ilya Sutskever，Geoffrey E. Hinton.

#### **1.1 论文概述**

AlexNet于2012年在论文《ImageNet Classification with Deep Convolutional Neural Networks》中由Hinton和他的学生Alex提出，用于2012年的ILSVRC竞赛。AlexNet在比赛中获得了第一名，并在top-5识别错误率上比当时的第二名低了将近10%.

当时的分类问题，主要是手工提取特征，然后放入如SVM等分类器进行分类。神经网络参数较大，当时的GPU性能不够优秀，所以没有被太多人重视。作者训练出了深度卷积神经网络用于图像分类任务，为一大首创，并且掀起了深度学习一大热潮，同时也引起了计算机视觉方法论的改变。

此外，AlexNet还应用了多GPU训练、局部响应归一化（LRN）、ReLU、dropout等具有创新性的方法，使得其在比赛中取得了优异的成绩。

#### **1.2 数据集**
AlexNet所使用的数据集为ImageNet.

ImageNet是一个拥有超过1500万张的已标定的高分辨率图像的数据集，大约有22,000个类别。


#### **1.3 模型讲解**
如图所示，由于采用双GPU协同训练，该网络结构图分为上下两部分，且两个GPU只在特定的层内通信。该模型一共分为8层，包括5个卷积层和3个全连接层。

<div align=center>
    <img src="./image/net.png" style="zoom:50%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">AlexNet网络结构图</div>
</div>

第一层：卷积层1，输入的图片大小为:224×224×3.卷积核的数量为96，大小为11×11×3，步长为4，不扩充边缘。经过处理后，输出的特征图大小为55×55×96.接着经过LRN和池化层，其中池化核大小为3×3，步长为2，最终得到27×27×96的特征图。

第二层：卷积层2，输入为上一层卷积的特征图。卷积核的数量为128×2=256，大小为5×5×48，步长为4，扩充值为2.经过处理后，输出的特征图大小为55×55×96.接着同样会经过LRN和池化层，其中池化核大小为3×3，步长为2，最终得到27×27×256的特征图。

第三层：卷积层3，输入为上一层的输出。卷积核的数量为192×2=384，大小为3×3×256，步长为1，扩充值为1.经过处理后，输出的特征图大小为13×13×384.第三层没有做LRN和池化。

第四层：卷积层4，输入为上一层的输出。卷积核的数量为192×2=384，大小为3×3×192，步长为1，扩充值为1.经过处理后，输出的特征图大小为13×13×384.第四层也没有做LRN和池化。

第五层：卷积层5，输入为上一层的输出。卷积核的数量为128×2=256，大小为3×3×192，步长为1，扩充值为1.经过处理后，输出的特征图大小为13×13×256.接着经过池化层，其中池化核大小为3×3，步长为2，最终得到6×6××256的特征图。

第六、七、八层：均是全连接层，每一层的神经元的个数为4096，最终经过softmax层，输出一个1000维的向量。全连接层中使用了ReLU和dropout。
#### **1.4 模型特点**

##### a) 非线性激活函数 ReLU:

标准的L-P神经元的输出一般使用$f(x) = tanh(x)$或$f(x) = (1+e^{-x})^{-1}$作为激活函数，而在训练阶段梯度衰减快慢方面，AlexNet所使用的ReLU函数，即$f(x) = max(0, x)$,比上述两个非线性饱和函数要快许多。效果详见下图：
<div align=center>
    <img src="./image/relu.png" style="zoom:50%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">ReLU（实线）-tanh（虚线）</div>
</div>

##### b) 多GPU训练:

在当时，单个GPU的内存限制了网络的训练规模，而采用多GPU训练可解决上述问题，提高AlexNet的训练速度。

但随着技术的发展，在绝大多数情况下，单个GPU的内存已经足以支持整个模型的训练，而多GPU训练也成为了AlexNet这个项目中最复杂的技术细节。此外，作者在此处提出的双GPU训练的可复现性差，所以在未来的几年内并没有被重视。

然而近些年随着体量更大的模型的出现，GPU瓶颈再次产生，这使得模型并行、分布训练（多GPU训练）又成为了大家解决GPU瓶颈的重要技术。

##### c) 局部响应归一化：

采用局部响应归一化(local response normalization，LRN)可将数据分布调整到合理的范围内，便于计算处理，从而提高泛化能力，避免过饱和。

具体公式如下：

$${b}_{x,y}^{i}={a}_{x,y}^{i}/({{k}+{α}{\sum_{j=max(0,i-n/2)}^{min(N-1,i+n/2)}}({a}_{x,y}^{i})^2})^{β}$$

但从现在的角度来看，LRN技术的作用并不大，而且在之后的训练中也并未得到广泛的使用。

##### d) 重叠池化层：

一般来说两个pooling是不重叠的，但是这里采用了一种对传统的pooling改进的方式，不仅可以提升预测精度，同时一定程度上可以减缓过拟合。


#### **1.5 降低过拟合**

##### a) 数据增强：

减少图像数据过度拟合的最简单也是最常见的方法是使用保留标签的变换人为地放大数据集。这里用了两种方式来实现数据增强：

i）随机区域提取：该方法在256×256的图片上随机提取一个224×224的区域用以训练，这使得数据集的大小变为原来的2048倍。

ii）RGB通道改变：该方法采用PCA的方式对RGB图像的channel进行了一些改变，使图像发生了一些变化，从而扩大了数据集。

##### b) dropout:

Dropout通过设置好的概率（50%），随机将某个隐藏层神经元的输出设置为0，因此这个神经元将不参与前向传播和反向传播，在下一次迭代中会根据概率重新将某个神经元的输出置0。简单来说就是在全连接层中去掉了一些神经节点，达到了防止过拟合的目的。但dropout的使用也让AlexNet的训练速度慢了一倍。Dropout相当于一个L2的正则化，只不过用了这种方式实现了L2正则化的功能。

#### **1.6 模型参数**

超参数：使用随机梯度下降法（SGD）训练模型，batch size = 128，momentum = 0.9，权重衰减 = 0.0005，学习率=0.01.

初始化：首先用均值为0，方差为0.01的高斯随机变量初始化权重参数，再将第2、4、5个卷积层和全连接起来的隐藏层的神经元偏置初始化为常数1，最后将剩余层的神经元偏置初始化为常数0.

学习率：在所有层上均使用0.01的学习率，验证集的错误率不再随当前学习率提高时，将学习率除以10.

### **2 案例实现**

#### **2.1 环境准备与数据读取**

本案例基于MindSpore-CPU版本实现，在CPU上完成模型训练。

环境需要mindspore和mindvision。

In [1]:
import mindspore
import mindspore.nn as nn

from mindspore import ops
from mindvision import dataset
from mindspore.dataset import vision

案例实现所使用的数据为CIFAR-10数据集。

CIFAR-10是一个用于识别普适物体的小型数据集。一共包含10个类别的 RGB 彩色图片。图片的尺寸为 32×32 ，数据集中一共有 50000 张训练图片和 10000 张测试图片。 CIFAR-10 的图片样例如图所示。

<div align=center>
    <img src="./image/cifar10.jpg", style="zoom:80%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">CIFAR-10图片样例</div>
</div>

本案例通过mindvision.dataset.Cifar10下载训练数据和测试数据，下载后的数据集会保存在设定好的路径。

In [2]:
# 从dataset下载训练数据
training_data = dataset.Cifar10(
    path="dataset_Cifar10",
    split="train",
    download=True,
)

# 从dataset下载测试数据
test_data = dataset.Cifar10(
    path="dataset_Cifar10",
    split="test",
    download=True,
)

# 数据下载完成后，获取数据集对象
train_dataset = training_data.dataset
test_dataset = test_data.dataset

# 查看数据集中包含的数据列名，用于dataset的预处理
train_dataset.column_names

[WARNING] ME(30384:26208,MainProcess):2022-11-06-17:31:39.811.112 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.
[WARNING] ME(30384:26208,MainProcess):2022-11-06-17:31:40.964.25 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.


['image', 'label']

#### **2.2 数据集处理**

我们对数据集进行转换，并拆分成多个batch。

In [3]:

# 这里我们设置batch大小，并定义图像所需要做的数据变换，包括‘Resize’、‘Rescale’、‘HWC2CHW’。
batch_size = 128

transforms = [
    vision.Resize((224, 224)),
    vision.Rescale(1.0 / 255.0, 0),
    vision.HWC2CHW()
]

# 创建训练集和测试集，并拆分成多个batch
train_dataset = train_dataset.map(transforms, 'image').batch(batch_size)
test_dataset = test_dataset.map(transforms, 'image').batch(batch_size)

for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

Shape of image [N, C, H, W]: (128, 3, 224, 224) Float32
Shape of label: (128,) Int32


#### **2.3 模型构建**

本案例实现中所构建的Alexnet模型结构与论文《ImageNet Classification with Deep Convolutional Neural Networks》中提出的结构大致相同，不同的是本案例直接使用CPU进行训练，而原论文中是使用的双GPU进行训练。因此，除了第一个卷积层外，第二到第四个卷积层都相较于原论文中的通道数进行了合并。模型结构如图所示。

<div align=center>
    <img src="./image/model_structure.png" alt="AlexNet-Structure" style="zoom:37%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">本案例使用的Alexnet模型结构</div>
</div>


MindSpore框架构建网络的流程与PyTorch类似，在定义模型类时需要继承Cell类，并重写__init__和construct方法。在此之前先定义了conv函数和fc_with_initialize函数用于自定义nn.Conv2d函数和nn.Dense函数。

具体的实现方式首先是在类中重写__init__方法，通过使用conv函数定义第一到第五个卷积层，使用mindspore.ops.operations.ReLU作为激活函数，通过nn.MaxPool2d定义池化，通过nn.Flatten定义Flatten，通过fc_with_initialize定义全连接层，通过nn.Dropout定义Dropout。之后在construct函数中构建前向网络。


In [4]:
def conv(in_channels, out_channels, kernel_size, stride=1, padding=0, pad_mode="valid", has_bias=True):
    """返回一个2D卷积模块"""
    return nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                     has_bias=has_bias, pad_mode=pad_mode)


def fc_with_initialize(input_channels, out_channels, has_bias=True):
    """获得一个全连接层"""
    return nn.Dense(input_channels, out_channels, has_bias=has_bias)


class AlexNet(nn.Cell):
    def __init__(self, num_classes=10, channel=1, dropout_ratio=0.5):
        """AlexNet模型
        Args:
            num_classes: 类别数
            channel    : 输入图像的通道数
        """
        super(AlexNet, self).__init__()
        self.channel = channel
        self.num_classes = num_classes
        self.dropout_ratio = dropout_ratio

        self.conv1 = conv(channel, 96, 11, stride=4, pad_mode='pad', padding=1, has_bias=True)
        self.conv2 = conv(96, 256, 5, pad_mode='pad', padding=2, has_bias=True)
        self.conv3 = conv(256, 384, 3, pad_mode='pad', padding=1, has_bias=True)
        self.conv4 = conv(384, 384, 3, pad_mode='pad', padding=1, has_bias=True)
        self.conv5 = conv(384, 256, 3, pad_mode='pad', padding=1, has_bias=True)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=3, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = fc_with_initialize(6400, 4096)
        self.fc2 = fc_with_initialize(4096, 4096)
        self.fc3 = fc_with_initialize(4096, num_classes)
        self.dropout = nn.Dropout(dropout_ratio)

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.conv5(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

#### **2.4 训练和测试函数的定义**

在模型训练中，一个完整的训练过程（step）需要实现以下三步：

1. 正向计算：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。
2. 反向传播：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。
3. 参数优化：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：

1. 正向计算函数定义。
2. 通过函数变换获得梯度计算函数。
3. 训练函数定义，执行正向计算、反向传播和参数优化。

In [5]:
def model_train(model, dataset, loss_fn, optimizer):
    """训练模型
    Args:
        model    : 模型
        dataset  : 数据集
        loss_fn  : 损失函数
        optimizer: 优化器
    """
    # 前向传播函数
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # 获得梯度函数
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # 进行每一步训练
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()

    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)
        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

测试模型函数

In [6]:
def model_test(model, dataset, loss_fn):
    """测试模型
    Args:
        model  : 模型
        dataset: 数据集
        loss_fn: 损失函数
    """
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0

    # 计算预测正确的样本个数
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    
    print(f"Test: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### **2.5 模型训练及评估**

训练过程需多次迭代数据集，一次完整的迭代称为一轮（epoch）。在每一轮，遍历训练集进行训练，结束后使用测试集进行预测。打印每一轮的loss值和预测准确率（Accuracy），可以看到loss在不断下降，Accuracy在不断提高。

这里TRAIN_EPOCH设置为30，IS_FIRST_TRAIN为True表示当前训练轮次为0，此时会从零开始训练，如果IS_FIRST_TRAIN为False则会导入模型参数，在之前训练过的模型的基础上继续训练。每一轮训练结束以后会保存一次模型。


In [7]:
# 本次训练的 epoch 数
TRAIN_EPOCH = 1

# 是否是第一次训练
IS_FIRST_TRAIN = False


def main():
    model = AlexNet(num_classes=10, channel=3)

    # 使用交叉熵作为损失函数
    loss_fn = nn.CrossEntropyLoss()

    # 使用随机梯度下降作为优化器，学习率设置为 0.01
    optimizer = nn.SGD(model.trainable_params(), 1e-2)

    # 如果不是第一次训练，那么导入之前保存的模型和之前累积训练的 epoch 数量
    if not IS_FIRST_TRAIN:
        param_dict = mindspore.load_checkpoint("model.ckpt")
        prev_epoch = param_dict['epoch']
        
        # 导入权重
        mindspore.load_param_into_net(model, param_dict)
    else:
        prev_epoch = 0

    # 开始训练
    for t in range(TRAIN_EPOCH):
        cur_epoch = t + prev_epoch + 1
        print(f"Epoch {cur_epoch}\n-------------------------------")

        model_train(model, train_dataset, loss_fn, optimizer)

        # 每次训练完一个 epoch 保存模型
        mindspore.save_checkpoint(model, "model.ckpt", append_dict={'epoch': cur_epoch})
        model_test(model, test_dataset, loss_fn)

训练模型

In [8]:
main()

Epoch 13
-------------------------------
loss: 2.047248  [  0/391]
loss: 1.830561  [100/391]
loss: 1.866250  [200/391]
loss: 2.000601  [300/391]
Test: 
 Accuracy: 33.4%, Avg loss: 1.838151 



#### **2.6 模型预测**

加载后的模型可以直接用于预测推理。

In [9]:
def predict(model, test_set):
    """预测
    Args:
        model   : 模型 
        test_set: 待预测的样本
    """
    model.set_train(False)
    for data, label in test_set:
        pred = model(data)
        predicted = pred.argmax(1)
        print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
        break


model = AlexNet(num_classes=10, channel=3)
param_dict = mindspore.load_checkpoint("model.ckpt")
mindspore.load_param_into_net(model, param_dict)
predict(model, test_dataset)

Predicted: "[7 9 6 3 9 0 6 9 3 3]", Actual: "[9 9 6 9 9 8 7 9 2 8]"


### **3 总结**

本案例基于MindSpore框架针对CIFAR-10数据集，完成了数据读取、数据集创建、Alexnet模型构建，进行了模型训练和评估，顺利完成了预测结果的输出。通过此案例进一步加深了对Alexnet模型结构和特性的理解，并结合MindSpore框架提供的文档和教程，掌握了利用Mindspore框架实现特定案例的流程，以及多种API的使用方法，为以后在实际场景中应用MindSpore框架提供支持。